In [2]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from tabulate import tabulate
from sklearn.decomposition import PCA
from sklearn.svm import LinearSVC
from sklearn.model_selection import train_test_split

# Set default Seaborn style
sns.set()

# Prevent pandas from wrapping colums to the next line
pd.set_option('expand_frame_repr', False)

# Increase the maximum number of rows shown
pd.set_option('display.max_rows', 1000)

pd.set_option('display.max_columns', 30)
pd.set_option('display.width', 1000)

plt.rcParams['figure.figsize'] = [20, 10]

print("All modules loaded successfully")

All modules loaded successfully
